# In class exercise 10

The purpose of the exercise is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 
    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import gensim
df=pd.read_csv("C:\Users\shiva\Downloads\archive (2).zip")
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from textblob import Word
nltk.download('wordnet')
from nltk.stem import PorterStemmer
st = PorterStemmer()
df = df[df['Reviews'].notnull()]
stop = stopwords.words('english')
df['after_punct_less'] = df['Reviews'].str.replace('[^\w\s].#','')
df['after_rm_stopwords'] =df['after_punct_less'].apply(lambda a: " ".join(a for a in a.split() if a not in stop))
df['after_numerics']=df['after_rm_stopwords'].str.replace('[0-9]','')
df['after_lowercasing'] =df['after_numerics'].apply(lambda a: " ".join(a.lower() for a in a.split()))

df['after_stemming']=df['after_lowercasing'].apply(lambda a: " ".join([st.stem(word) for word in a.split()]))
df['cleaned_text'] = df['after_stemming'].apply(lambda a: " ".join([Word(word).lemmatize() for word in a.split()]))

In one paragraph, please compare K means, DBSCAN and Hierarchical clustering. 

In [ ]:
df['cleaned_text'].head()

In [ ]:
df_1=(df.sample(n=4000)).reset_index()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf = tfidf_vect.fit_transform(df_1['cleaned_text'].values)
tfidf.shape

In [ ]:
from sklearn.cluster import KMeans
model_tf = KMeans(n_clusters = 4, n_jobs = -1,random_state=89)
model_tf.fit(tfidf)

In [ ]:
labels_tf = model_tf.labels_
cluster_center_tf=model_tf.cluster_centers_

In [ ]:
terms1 = tfidf_vect.get_feature_names()
terms1[1:4]
df1 = df_1
df1['Tfidf Clus Label'] = model_tf.labels_
df1.head(4)

In [ ]:
df1.groupby(['Tfidf Clus Label'])['cleaned_text'].count()

In [ ]:
print("Highest terms per cluster:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(1,4):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :4]:
        print(' %s' % terms1[ind], end='')
        print()

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import gensim

In [ ]:
i=0
list_of_sent=[]
for sent in df_1['cleaned_text'].values:
    list_of_sent.append(sent.split())

In [ ]:
w2v_model=gensim.models.Word2Vec(list_of_sent,size=90, workers=4)
import numpy as np
sent_vectors = [];
for sent in list_of_sent: 
    sent_vec = np.zeros(90) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)
sent_vectors.shape

In [ ]:
minPts = 2 * 100
def lower_bound(nums, target): 
    l, r = 0, len(nums) - 1
    while l <= r: 
        mid = int(l + (r - l) / 2)
        if nums[mid] >= target:
            r = mid - 1
        else:
            l = mid + 1
    return l

def compute200thnearestneighbour(a, data): 
    dists = []
    for val in data:
        dist = np.sum((a - val) **2 ) 
        if(len(dists) == 200 and dists[199] > dist): 
            l = int(lower_bound(dists, dist)) 
            if l < 200 and l >= 0 and dists[l] > dist:
                dists[l] = dist
        else:
            dists.append(dist)
            dists.sort()
    
    return dists[199]

In [ ]:
twohundrethneigh = []
for val in sent_vectors[:1000]:
    twohundrethneigh.append( compute200thnearestneighbour(val, sent_vectors[:1000]) )
twohundrethneigh.sort()

In [ ]:
mob_vectors.shape

In [ ]:
plt.figure(figsize=(14,4))
plt.title("Finding the correct Eps hyperparameter by using elbow method")
plt.plot([a for a in range(len(twohundrethneigh))], twohundrethneigh)
plt.xlabel("Number of points")
plt.ylabel("Distance of 200th Nearest Neighbour")
plt.show()

In [ ]:
model = DBSCAN(eps = 4, min_samples = minPts, n_jobs=-1)
model.fit(sent_vectors)
df_1['AVG-W2V Clus Label'] = model.labels_
df_1.head(3)

In [ ]:
import scipy
from scipy.cluster import hierarchy
dendro=hierarchy.dendrogram(hierarchy.linkage(sent_vectors,method='ward'))
plt.axhline(y=35)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')  
Agg=cluster.fit_predict(sent_vectors)

In [ ]:
df_1['AVG-W2V Clus Label'] = cluster.labels_
df_1.head(3)
df_1.groupby(['AVG-W2V Clus Label'])['cleaned_text'].count()

In [ ]:
for i in range(4):
    print("3 reviews of given cluster ", i)
    print("-" * 70)
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][0]]['cleaned_text'])
    print('\n')
    print(df_1.iloc[df_1.groupby(['AVG-W2V Clus Label']).groups[i][1]]['cleaned_text'])
    print('\n')
    print("_" * 70)

In [ ]:
#You can write you answer here.
Cluster analysis is the task of grouping a set of objects in such a way that objects in the same group. 
It is a main task of exploratory data mining, and a common technique for statistical data analysis, used in many fields, 
including pattern recognition, image analysis and machine learning. The K-means clustering algorithm is used to find groups which have not been 
explicitly labeled in the data. This can be used to confirm business assumptions about what types of groups exist or
 to identify unknown groups in complex data sets. To use k means clustering the first step we need to do is select the number of clusters 
 Second, Select k random points from the data as centroids and then Assign all the points to the closest cluster centroid.
Recompute the centroids of newly formed clusters and then repeat the third and 4th steps untill the requirement is met
